# Alpha-AGI Business v1 • Colab Notebook

Run the introductory **Alpha-Factory** business demo. Works offline
or upgrades automatically when `OPENAI_API_KEY` is set.

## 0 · Runtime check

In [ ]:
!nvidia-smi -L || echo 'GPU not detected - running on CPU'

## 1 · Clone repo & install dependencies

In [ ]:
%%bash
set -e
if [ ! -d AGI-Alpha-Agent-v0 ]; then
  git clone --depth 1 https://github.com/MontrealAI/AGI-Alpha-Agent-v0.git
fi
cd AGI-Alpha-Agent-v0
pip -q install openai_agents fastapi uvicorn gradio

## 2 · (Optional) Configure your OpenAI API key

In [ ]:
import os, getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('Enter OpenAI API key (leave blank for offline): ')

## 3 · Launch orchestrator

In [ ]:
import subprocess, re, queue, threading, sys, time
from IPython.display import display, IFrame

root = 'AGI-Alpha-Agent-v0/alpha_factory_v1/demos/alpha_agi_business_v1'
proc = subprocess.Popen([sys.executable, '-m', 'alpha_agi_business_v1'], cwd=root, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)

q = queue.Queue()

def _tail():
    for line in proc.stdout:
        print(line, end='')
        m = re.search(r'http://localhost:(\d+)/docs', line)
        if m:
            q.put(int(m.group(1)))

threading.Thread(target=_tail, daemon=True).start()
port = q.get()
print(f'⏳ Waiting for REST docs on port {port}...')
for _ in range(40):
    try:
        import requests
        if requests.get(f'http://localhost:{port}/healthz').status_code == 200:
            break
    except Exception:
        time.sleep(0.2)

iframe = IFrame(src=f'http://localhost:{port}/docs', width='100%', height=600)
display(iframe)

## 4 · Programmatic API call

In [ ]:
import requests
print('Available agents:', requests.get('http://localhost:8000/agents').json())

## 4b · Trigger Execution Agent

In [ ]:
import requests
requests.post('http://localhost:8000/agent/alpha_execution/trigger')
print('execution triggered')

## 5 · OpenAI Agents bridge

In [ ]:
%%bashpython openai_agents_bridge.py >/tmp/bridge.log 2>&1 &sleep 2tail -n 5 /tmp/bridge.log

In [ ]:
# 5c · Query recent alpha via OpenAI Agents bridge
import requests
resp = requests.post('http://localhost:5001/v1/agents/business_helper/invoke', json={'action':'recent_alpha'})
if resp.status_code == 200:
    try:
        print('recent alpha:', resp.json())
    except ValueError as e:
        print('Error decoding JSON response:', e)
else:
    print(f'Error: Received status code {resp.status_code} with response: {resp.text}')

In [ ]:
# 5d · Submit a custom job via the bridge
job = {'agent': 'alpha_execution', 'symbol': 'AAPL', 'qty': 1}
resp = requests.post('http://localhost:5001/v1/agents/business_helper/invoke', json={'action':'submit_job', 'job': job})
print('job submission:', resp.text)

## 6 · Graceful shutdown

In [ ]:
proc.terminate(); print('✅ Orchestrator stopped')